# Emotion Propagation in Online Social Networks
##### Code Workflow
This is the framework code for the '*Emotion Propagation in Online Social Networks*' project. <br>
You can find the associated paper here.

## Data Collection and Pre-Processing

### Setup

#### Imports

In [ ]:
# Custom modules path
import sys; sys.path.append('./libraries')
import warnings; warnings.filterwarnings('ignore')
# Custom modules
from utils import *
# Basic libraries
import pandas as pd
import numpy as np
from collections import Counter
from datetime import timedelta
import re
# Network libraries
import networkx as nx
from scipy.sparse import csgraph
import network_distance as nd
# Statistic libraries
from scipy.stats import entropy, pearsonr, spearmanr, ks_2samp, kruskal, tukey_hsd
# NLP libraries
import tweetnlp
from nltk.corpus import stopwords
# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

#### Filepaths & Constants

Relative paths to the repository.

In [ ]:
root = '../data/'
raw_path = root + '0- Raw collected data/'
processed_path = root + '1- Processed data/'
finetuning_path = root + '2- NLP/1- Fine Tuning'
emotions_path = root + '2- NLP/2- Emotions/'
propagation_path = root + '3- Propagation/'

Switches to process data from scratch, or load the processed data. <br>
For looking at results, suffices to set all switches to False. <br>
For corroboration of the steps, suffices to set the relevant switches to True. <br>
For complete replication of the results from the raw collected data, set everything to True. NOTE: We do not recommend this, since the classification process takes a very long time. We therefore recommend running the classification in a High Perfomance Cluster.

In [ ]:
# SWITCHES : 
process = False                 # To process everything from Raw Data or load Processed Data
classify = False                 # To classify and extract tweets emotions or load Classified Data
propagate = False                # To calculate propagation or load Propagation Data
access_to_all = False

A few variable settings.

In [ ]:
# SETTINGS
# Set of Networks
networks = {
    "us_congressmen": {}, 
    "journalists": {}, 
    "universities": {}
    }

# Set of emotions (and their associated color)
emotions = {
    "joy": "gold",
    "optimism": "lawngreen",
    "anger": "crimson",
    "sadness": "mediumslateblue"
    }

# Set of Datasets collected
datasets = ["user_info", "tweets", "follows"]

### Load raw data

Data was collected using [Twitter Lists](https://help.twitter.com/en/using-twitter/twitter-lists). This allowed us to collect data on specific users from a determined "community". The collected "Twitter Communities" are from three different domains:
- US Congressmen: Current and former (previous period) US Congressmen (Senators and Representatives);
- Journalists:  Journalists and staff from Wall Street Journal, NY Times, NY Post, Washington Post and LA Times;
- Universities: Researchers and staff from Cambridge University, NY University and Princeton University  

For each of those communities, three datasets were collected:
- User Info: includes Twitter User ID, Display Name, Username (handle), Public Metrics, etc;
- Tweets: includes tweets for the period Jan-2022 to Feb-2023 for each user
- Follows: the list of people following by each of the users in User Info. This is used to construct the network

We'll organize our networks in a dictionary containing each dataset.

In [ ]:
if access_to_all:
    print(">>> LOADED DATA <<<")
    for net in networks:

        if not process:
            # We don't save 'follows' as processed
            datasets = datasets[:2]
            path = processed_path

        else:
            # If we process everything from raw
            path = raw_path + f"{net}/"

        for ds in datasets:

            networks[net][ds] = pd.read_csv(path + f"DF-{net}-{ds}.csv", encoding="utf-8", dtype={"id": str, "author_id": str, "mainUserID": str, 
                                                "retweeted": str, "possibly_sensitive": str})

            # Filter only tweets from before 23/12
            if ds == "tweets":
                #networks[net][ds].loc[:,"created_at"] = pd.to_datetime(networks[net][ds].created_at).dt.date
                # We need to convert to Datetime format to truncate miliseconds
                networks[net][ds].loc[:,"created_at"] = pd.to_datetime(networks[net][ds].created_at, format="%Y-%m-%d %H:%M:%S")

            print(f"{'RAW' if process else 'PROCESSED'} DATA // Network: {net} - {ds} - Shape: {networks[net][ds].shape}")

### Pre-Process Tweets

#### Replace Retweets' text by original text
By default, when a Tweet is a Retweet, Twitter appends the prefix RT and shortens the text (possibly to save space). Nevertheless, the id of the original tweet is also returned. From this ID we restore the original text of the tweet and replace the shorten version by the original one.

In [ ]:
if process:
    
    for net in networks:

        # Read auxiliary retweet data and rename columns to match Tweets DF
        aux_rt = pd.read_csv(raw_path + f"{net}/DF-{net}-aux_retweets.csv", encoding="utf-8", dtype={"id": str})
        aux_rt.columns = ["retweeted", "text"]

        # Split Tweets DF in original tweets and retweets
        originals = networks[net]["tweets"][networks[net]["tweets"].retweeted.isna()]
        retweets = networks[net]["tweets"][~networks[net]["tweets"].retweeted.isna()]

        # Merge the retweets with the aux data to get the complete text
        retweets = retweets.merge(aux_rt, on= "retweeted", how="inner") # If inner, we are discarding RT that haven't been found

        # Replace the RT shorten text for the original text, drop aux col, and rename back to 'text'
        retweets["text_x"] = retweets.text_y
        retweets.drop("text_y", axis=1, inplace=True)
        retweets.rename(columns={'text_x':'text'}, inplace=True)

        # Put back together the Tweets Dataset
        networks[net]["tweets"] = pd.concat([originals, retweets])
        networks[net]["tweets"].sort_index(inplace=True, ignore_index=True)

#### Filter Tweets

The data-cleaning process of the tweets include:
- Filter out non-English tweets: The NLP model used is trained only on English tweets, to preserve the quality of the analysis;
- Remove mentions: Mentions in Tweets are removed, to reduce noise in the text;
- Remove URLs: URLs are also removed, for the same reasons as above;
- Remove undefined characters: Some tweets contain undefined non-parsed characters that were removed to preserve the quality of the data


In [ ]:
if process:
    
    for net in networks:

        # Filter our non-English tweets
        networks[net]["tweets"] = tweetFilter(networks[net]["tweets"])

### Prepare Network

Check overall yearly activity for different window sizes, using the raw data.

In [ ]:
if process:

    # Style
    windows = {
        3: ["dotted", "."],
        7: ["dashed", "^"],
        10: ["solid", "o"]
    }
    
    # Check for different window sizes
    for w in windows:
        
        # Calculate active users
        for net in networks:

            # We need to check if the window will leave the last gap unfinished
            temp = networks[net]["tweets"].loc[:,["created_at", "author_id"]].groupby(pd.Grouper(key="created_at", freq=f"{w}d")).nunique("author_id").reset_index()
            if ((networks[net]["tweets"].created_at.max() - pd.to_datetime("2022-01-01", format="%Y-%m-%d %H:%M:%S")).days + 1) % w > 0:
                temp = temp.iloc[:-1]
            temp["author_id"] = temp["author_id"] / networks[net]["tweets"].author_id.nunique() * 100
            sns.set(rc={'figure.figsize':(16,4)})
            sns.set_style("ticks")
            sns.lineplot(temp, x="created_at", y="author_id", linestyle=windows[w][0], marker=windows[w][1])
            sns.despine()
            plt.grid(axis='both', color='grey', linestyle='dashed', linewidth=0.5)
            plt.xlabel("")
            plt.ylabel("")
            plt.ylim(20,100)
        plt.show()

Check activity (number of tweets) per weekday.

In [ ]:
if process:

    # For each network
    for net in networks:

        print(f"Network: {net.upper()}")
        # Use a copy of the tweets dataset
        temp = networks[net]["tweets"].copy()
        temp["created_at"] = pd.to_datetime(temp["created_at"])
        # Extract the weekday number and name for the dates
        temp["#"] = temp["created_at"].apply(lambda x: x.weekday())
        temp["Day of Week"] = temp["created_at"].apply(lambda x: x.day_name())

        # Calculate the number of tweets posted per day of the week
        day_activity = temp.groupby(["#", "Day of Week"]).size().reset_index()
        day_activity.columns = ["#", "Week Day", "Activity"]
        # Calculate the percentage of activity
        day_activity["%"] = round(day_activity.Activity / day_activity.Activity.sum() * 100,2)

        # Plot settings
        sns.set(rc={'figure.figsize':(16,4)})
        sns.set_style("ticks")
        sns.lineplot(day_activity, x="Week Day", y="%", palette=["blue", "orange", "green"], marker="s")
        sns.despine()
        plt.grid(axis='y', color='grey', linestyle='dashed', linewidth=0.5)
        plt.xlabel("Weekday")
        plt.ylabel("% Activity")

#### Remove users with large inactivity

We want to remove those users that are not active enough to be considered in the experiment. For this, a baseline threshold is defined:
* Window: 7 days. To be considered active, we'll evaluate the amount of tweets the user has in a period of seven days.
* Activity Threshold: 4 tweets. In the period of 7 days, the user needs to have tweeted at least 4 times.
* Max Gap: 1. If the user doesn't reach this minimum activity threshold in one given week, the "expressed emotion" can be interpolated. If the user violates this minimum activity baseline for 2 or more consecutive weeks, then the user is removed from the experiment.

##### Parameters

In [ ]:
# Define parameters to consider minimum activity of users
window = "7d"               #-> The window for considering the emotional status of a node
activity_threshold = 4      #-> The minimum number of tweets an user needs to be considered "active" during the time window
max_gap = 1                 #-> The maximum consecutive gap allowed for users to be "inactive"

In [ ]:
if process:
    
    for net in networks:

        # We need to convert to Datetime format to truncate miliseconds
        networks[net]["tweets"].created_at = pd.to_datetime(networks[net]["tweets"].created_at, format="%Y-%m-%d %H:%M:%S")

        # Get subset of active users
        act_urs = getActiveUsers(networks[net]["tweets"], window, max_gap, activity_threshold)
        networks[net]["user_info"] = networks[net]["user_info"].loc[networks[net]["user_info"].id.isin(act_urs)]
        networks[net]["follows"] = networks[net]["follows"].loc[networks[net]["follows"].mainUserID.isin(act_urs)]

#### Create Network

- Remove users not following each other
- Remove nodes not in largest component
- Compute edge weights
- Create edgelist and graph

In [ ]:
# For each network
for net in networks:
    
    if process:

            # Use edgeListMaker to create the Edgelist from users following each other
            edgelist = edgeListMaker(networks[net]["user_info"], networks[net]["follows"], networks[net]["tweets"], save_edgelist=True, \
            edgelist_path=processed_path + f"EDGELIST-{net}-undirected.csv")
            networks[net]["edgelist"] = edgelist
            # We create the Graph from the Edgelist
            G = nx.from_pandas_edgelist(edgelist)
            networks[net]["graph"] = G

            # We identify the nodes to reduce the datasets to only those nodes, and store them
            nodes = list(G.nodes())
            networks[net]["user_info"] = networks[net]["user_info"].loc[networks[net]["user_info"].id.isin(nodes)]
            networks[net]["follows"] = networks[net]["follows"].loc[networks[net]["follows"].mainUserID.isin(nodes)]
            networks[net]["tweets"] = networks[net]["tweets"].loc[networks[net]["tweets"].author_id.isin(nodes)]
        
    # Else read from processed data
    else:
        networks[net]["graph"] = nx.read_edgelist(processed_path + f"EDGELIST-{net}-undirected.csv", comments="source", delimiter=",", data=(("weight", float),), encoding='utf-8')

Force all members of US Congress to adopt a bipartisan disposition

In [ ]:
if access_to_all:
    networks["us_congressmen"]["user_info"].loc[networks["us_congressmen"]["user_info"]["party"] != "Republican", "party"] = "Democratic"

#### Save reduced process datasets

In [ ]:
if process:

    for net in networks:

        ## TWEETS ##
        # We only need these columns
        keep_cols = ["author_id", "id", "text", "created_at", "retweeted"]
        # We only need to know if the tweet is a retweet (transform to bool to save space)
        networks[net]["tweets"]["retweeted"] = networks[net]["tweets"]["retweeted"].apply(lambda x: False if type(x) != str else True)
        networks[net]["tweets"] = networks[net]["tweets"][keep_cols]
        # Save to Processed Data
        networks[net]["tweets"].to_csv(processed_path + f"DF-{net}-tweets.csv", encoding="utf-8", index=False)

        ## USER INFO ##
        networks[net]["user_info"].drop(["created_at", "protected", "verified", "followers_count", "following_count", "listed_count"], axis=1, inplace=True)
        # Save to Processed Data
        networks[net]["user_info"].to_csv(processed_path + f"DF-{net}-user_info.csv", encoding="utf-8", index=False)

#### Show Networks basic properties

In [ ]:
# Partition for different networks
partition = {
    "us_congressmen": "party",
    "journalists": "newspaper",
    "universities": "university"
    }
if access_to_all:
    for net in networks:
        print(f"Network -> {net}")
        # Extract list of nodes
        nodes = list(networks[net]["graph"].nodes())
        communities = []
        # Construct the set of communities given their respective partition
        for p in networks[net]["user_info"][partition[net]].unique():
            # Nodes for a given partition of the network
            p_nodes = networks[net]["user_info"].loc[(networks[net]["user_info"][partition[net]] == p) & (networks[net]["user_info"]["id"].isin(nodes))].id
            # Append the partition set to the list of partitions
            communities.append(set(p_nodes))

        GraphProperties(networks[net]["graph"], communities=communities)

## Emotion Recognition

### Fine Tuning

The chosen NLP model to extract the emotion from tweets is [TweetNLP](https://tweetnlp.org/), a roBERTa-base model trained on tweets, and further fine-tuned for the task of emotion extraction. 

In [ ]:
# Let's load the TweetNLP Training set
dataset, label_to_id = tweetnlp.load_dataset("emotion")

#### Label distributions

For train set in TweetNLP Baseline fine tuning:

In [ ]:
# Let's look the label distribution
total = len(dataset['train']['label'])
print(f"Total training samples: {total}\n")
print("Training dataset label distribution:\n")

for label, i in label_to_id.items():
    nr = len([x for x in dataset["train"]["label"] if x == i])
    print(f"{label.upper()}: {nr} ({nr/total:.2%})")

For annotation samples:

In [ ]:
if access_to_all:
    # Let's load our annotations
    samples, labels = load_annotations(finetuning_path + '/annotations/')

    # Let's look the label distribution for our annotations
    total = len(labels)
    print(f"Total annotated samples: {total}\n")
    print("Annotated dataset label distribution:\n")

    for label in ["undefined"] + list(label_to_id.keys()):
        nr = len([x for x in labels if x == label])
        print(f"{label.upper()}: {nr} ({nr/total:.2%})")

Label distribution of samples used in fine tuning process:

In [ ]:
if access_to_all:
    # Let's look the label distribution for our custom fine tuning
    train = pd.read_csv(finetuning_path + '/all/train.csv', encoding="utf-8")
    total = train.shape[0]
    print(f"Total training samples: {total}\n")
    print("Training dataset label distribution:\n")

    for label, i in label_to_id.items():
        nr = train[train.label == i].shape[0]
        print(f"{label.upper()}: {nr} ({nr/total:.2%})")

#### Classify using Fine Tuned TweetNLP

Given the large amount of tweets to classify (2.060.983), this stage was performed in ITU's High Performance Cluster for approximately 206 hs. <br>
The example set up follows here:

In [ ]:
if access_to_all:
    if classify:
    
    # Load Finetuned model
    model = tweetnlp.load_model('emotion', model_name="./nlp_model/models/all")

    # Set a batch size according to resources
    batch_size = 10000

    # Classify tweets for each network
    for net in networks:
    
        # We need to split tweet dataset in batches
        classified_samples = []

        # Read tweet dataset
        tweets = pd.read_csv(f"./data/1- Processed data/DF-{net}-tweets.csv", encoding="utf-8")

        # Get number of batches needed
        batches =  tweets.shape[0] // batch_size + 1

        # Predict by batches
        for batch in range(0, batches):

            # If it's the last batch, slice to the end
            if batch == batches-1:
                tweets_subset = tweets.iloc[(batch*batch_size):]

            # Otherwise slice to the batch
            else:
                tweets_subset = tweets.iloc[batch*batch_size:(batch*batch_size)+batch_size]

            # Predict on text
            result = model.predict(tweets_subset.text, return_probability = True)
            result = pd.DataFrame(result)

            # Extract the predictions and calculate the entropy
            joy = []
            optimism = []
            anger = []
            sadness = []

            for ix, row in result.iterrows():
                joy.append(row["probability"]["joy"])
                optimism.append(row["probability"]["optimism"])
                anger.append(row["probability"]["anger"])
                sadness.append(row["probability"]["sadness"])

            result["joy"] = joy
            result["optimism"] = optimism
            result["anger"] = anger
            result["sadness"] = sadness
            result.drop("probability", axis=1, inplace=True)
            result["nEntropy"] = result[["joy", "optimism", "anger", "sadness"]].apply(entropy, axis=1)
            result["nEntropy"] = result.nEntropy / (np.log(4))

            # Append the batch results
            classified_samples.append(result)

        # Concat all results in one DF
        classified_samples = pd.concat(classified_samples)

        # Keep only User ID, Creation Date and Classification results
        classified_samples = classified_samples[["author_id", "created_at", "retweeted", "joy", "optimism", "anger", "sadness", "nEntropy"]]

        # Save to file
        classified_samples.to_csv(f"{emotions_path}{net}-emotions.csv", encoding="utf-8", index=False)

We can now load the classified emotions per tweet, user and date.

In [ ]:
for net in networks:

    networks[net]["emotions"] = pd.read_csv(f"{emotions_path}{net}-tweets-emotions.csv", encoding="utf-8", dtype={"author_id": str})

### Transform uncertain predictions into undefined, aggregate emotions and interpolate missing emotions in gaps

##### First, let's look at the proportion of labeled samples given our entropy cutoff

In [ ]:
# Decided entropy cutoff
entropy_cutoff = 0.07596

print(f"With entropy cutoff at {entropy_cutoff}")
for net in networks:
    #np.percentile(networks[net]["emotions"].nEntropy, np.linspace(0,100,11))
    prop = networks[net]["emotions"][networks[net]["emotions"].nEntropy <= entropy_cutoff].shape[0] /networks[net]["emotions"].shape[0]
    print(f"Network {net}: we keep {prop:.2%} labeled")

In [ ]:
for net in networks:

    # Scrap values from undefined emotions, given an entropy threashold
    networks[net]["emotions-undefined"] = undefine(networks[net]["emotions"], entropy = entropy_cutoff)

    # Remove noise from low probabilities
    networks[net]["emotions-undefined"] = remove_noise(networks[net]["emotions-undefined"])

    # Aggregate the emotions by week
    networks[net]["emotions-aggregated"] = find_weekly_values(networks[net]["emotions-undefined"])
    networks[net]["emotions-aggregated"].rename(columns={"agg_optimism": "optimism", "agg_joy": "joy", "agg_anger": "anger", "agg_sadness": "sadness"}, inplace=True)

    # Interpolate the gaps
    dfs = []
    for author in networks[net]["emotions-aggregated"].author_id.unique():

        df = networks[net]["emotions-aggregated"].loc[networks[net]["emotions-aggregated"].author_id == author]
        df = interpolate_values(df, activity_threshold = activity_threshold, emotion_idx = [2,3,4,5], act_idx = 6)
        dfs.append(df)

    networks[net]["emotions-interpolated"] = pd.concat(dfs)

#### Testing for interpolation accuracy
To ensure that the interpolation process has not changed the nature of the data, we compute a few statistics:
- Compare average emotion fluctuationa between time steps for all users in all networks
- Evaluate if the distributions are different with a KS test
- Calculate MAE, RMSE and R2 for data before and after interpolation

In [ ]:
# Check the emotion fluctuation between timesteps for all users
before_avg = []
after_avg = []

for net in networks:

    print(f"{net.upper()}")

    # Keep only continuos week rows
    continous = networks[net]["emotions-aggregated"][networks[net]["emotions-aggregated"]["weekly_tweet_values"] >= 4]
    # Calculate the gap to ensure we only keep continuos weeks
    continous["gap"] = continous.week.diff().fillna(1)
    continous = continous[continous.gap < 2]

    # For each emotion column
    for e in emotions:

        # We extract the emotion vector before and after interpolation
        original_data = networks[net]["emotions-aggregated"][e]
        interpolated_data = networks[net]["emotions-interpolated"][e]

        # We calculate the critical value for the KS test
        critical_value = 1.358 / (len(original_data) + len(interpolated_data))**0.5  # for alpha = 0.05
        print(f"{e.capitalize()} (critical value: {critical_value:.3f})") 

        # Perform KS test
        stat, p_value = ks_2samp(original_data, interpolated_data)

        # Calculate metrics on dataset difference 
        # Mean Absolute Error (MAE)
        mae = np.mean(np.abs(interpolated_data - original_data))
        # Root Mean Squared Error (RMSE)
        rmse = np.sqrt(np.mean((interpolated_data - original_data) ** 2))
        # Coefficient of Determination (R-squared)
        total_variation = np.sum((original_data - np.mean(original_data)) ** 2)
        residual_variation = np.sum((interpolated_data - original_data) ** 2)
        r_squared = 1 - (residual_variation / total_variation)

        # compare p-value to significance level (e.g., 0.05)
        if stat > critical_value:
            print(f"SAME DISTRIBUTION (stat: {stat:.3f} - p-value: {p_value:.3f})")
        else:
            print(f"DIFFERENT DISTRIBUTION (stat: {stat:.3f} - p-value: {p_value:.3f})")

        print(f"Mean Absolute Error: {mae:.3f}")
        print(f"Root Mean Squared Error: {rmse:.3f}")
        print(f"R-squared: {r_squared:.3f}")
    
        # For each author separately
        for a in continous.author_id.unique():
            # Calculate the fluctuation before and after interpolation
            before_interpolation = continous.loc[continous.author_id == a, e].diff().fillna(0)
            after_interpolation = networks[net]["emotions-interpolated"].loc[networks[net]["emotions-interpolated"].author_id == a, e].diff().fillna(0)
            # Calculate their emotion fluctuation
            before_avg.append(sum(abs(before_interpolation)) / before_interpolation.shape[0])
            after_avg.append(sum(abs(after_interpolation)) / after_interpolation.shape[0])

print(f"Before interpolation, the avg. fluctuation for all users in all networks is {np.mean(before_avg):.5f}")
print(f"After interpolation, the avg. fluctuation for all users in all networks is {np.mean(after_avg):.5f}")

## Emotion Presence in the Networks

##### Boxplots

In [ ]:
# For each network
for net in networks:

    print(net.upper())

    # Melt emotion columns to show boxplots
    temp = networks[net]["emotions-interpolated"].melt(value_vars=["optimism", "joy", "anger", "sadness"])

    # Plot settings
    sns.set(rc={'figure.figsize':(16,2)})
    sns.set_style("ticks")
    sns.boxplot(data = temp, y="variable", x="value", palette=emotions)
    plt.xlim(-0.01,1.01)
    plt.xlabel("")
    plt.ylabel("")
    plt.grid(axis="x", linestyle="dotted", color="grey", linewidth=0.5)
    sns.despine()
    plt.show()

##### Emotion presence avg, std, and quartiles

In [ ]:
for net in networks:
    print(f"NETWORK {net.upper()}")
    temp = networks[net]["emotions-interpolated"][emotions].describe()
    temp.drop("count", inplace=True)
    display(temp.T)

#### Testing for significant differences between the networks
We first use a Levene test to see if the data sets are suitable for an ANOVA test.

In [ ]:
for net in networks:
    print(f"Network {net.upper()}")
    for e in emotions:
        stat, p_value = kruskal(*[networks[net]['emotions-interpolated'][e] for net in networks])
        print(f"Emotion {e.capitalize()} - P-Value: {p_value} - {p_value / (0.05 * 4)}")

### Looking at emotions per network sub-groups

In [ ]:
if access_to_all:
    # Since the groups in each network are called something different, they are coded seperatly
    networks['us_congressmen']['Group'] = (networks['us_congressmen']['emotions-interpolated'].join(
        networks['us_congressmen']['user_info'][['id', 'party']].set_index('id'), on = 'author_id').rename(columns = {'party' : 'Group'}))

    networks['journalists']['Group'] = (networks['journalists']['emotions-interpolated'].join(
        networks['journalists']['user_info'][['id', 'newspaper']].set_index('id'), on = 'author_id').rename(columns = {'newspaper' : 'Group'}))

    networks['universities']['Group'] = (networks['universities']['emotions-interpolated'].join(
        networks['universities']['user_info'][['id', 'university']].set_index('id'), on = 'author_id').rename(columns = {'university' : 'Group'}))

    for net in networks:
        networks[net]['sub_group'] = pd.melt(networks[net]['Group'].groupby(['week', 'Group']).mean().reset_index(),
            id_vars=['week', 'Group'], value_vars=["optimism", "joy", "anger", "sadness"])

    for net in networks:
        networks[net]['group_dict'] = {}
        for group in networks[net]['sub_group']['Group']:
            networks[net]['group_dict'][group] = networks[net]['Group'].author_id[networks[net]['Group'].Group == group]
        networks[net].pop("Group")
        networks[net].pop("group_dict")

In [ ]:
if access_to_all:
    sns.set_style('ticks')
    sns.set(rc={'figure.figsize':(16,4)})

    for net in networks:
        sns.barplot(data = networks[net]['sub_group'], x = 'variable', y = 'value', hue = 'Group', palette='rocket')
        plt.grid(axis = 'y', linestyle = 'dotted', color = 'gray', linewidth = 0.5)
        sns.despine()
        plt.show()

#### Significance test on the mean difference

To test if there is a significant difference between the sub-groups within each network, we compute the mean emotional state for users within the same sub-group for all weeks. Since the variance is significantly different within groups, we perform a Kruskal-Wallis test. 

In [ ]:
if access_to_all:
    print("KRUSKAL-WILLIS - TEST")
    for net in networks:
        for e in emotions:
            temp = networks[net]['sub_group'][networks[net]['sub_group']['variable'] == e]
            stat, p_value = kruskal(*[temp['value'] [temp.Group == group] for group in temp.Group.unique()])
        print(f"For network {net.upper()}: P-Value: {p_value:.5f}")

    print("\nTUKEY's HSD - TEST")

    for net in networks:
        print(f"Network {net.upper()}")
        for e in emotions:
            temp = networks[net]['sub_group'][networks[net]['sub_group']['variable'] == e]
            res = tukey_hsd(*[temp['value'] [temp.Group == group] for group in temp.Group.unique()])
            p_value = res.pvalue.flatten()[res.statistic.argmax()]
            print(f"{e.capitalize()} - P-Value: {p_value:.5f}")

## Measuring Propagation

### Prepare networks:
- Calculate the inverse of the Laplacian matrix
- Melt the emotion columns to one 'Emotion' column

In [ ]:
if propagate:
    for net in networks:

        # Needed graph calculations
        Q = nx.adjacency_matrix(networks[net]["graph"], weight = "weight")
        Q = csgraph.laplacian(Q, normed = False)
        Q = Q.todense()
        Q = Q.astype(float)
        Q = np.linalg.pinv(Q)

        networks[net]["Q"] = Q

        df = pd.melt(networks[net]["emotions-interpolated"], id_vars=['author_id', 'week'], value_vars=['joy', 'anger', 'sadness', 'optimism'], var_name="emotion")

        # List of users and weeks with amount of emotion
        networks[net]["emotions-weekly"] = df

### Calculate distances for emotions between weeks

In [ ]:
weeks = 51

for net in networks:

   if propagate:

      # List to store distances
      distances = []
      distances_norm = []

      # For each emotion
      for emotion in networks[net]["emotions-weekly"].emotion.unique():

         # Isolate the emotion
         df_emotion = networks[net]["emotions-weekly"][networks[net]["emotions-weekly"]["emotion"] == emotion]

         week_array = []
         # For each week
         for w in range(weeks):
            # Isolate emotions for a given week
            df_week = df_emotion[df_emotion.week == w]
            _ = {}
            # For each node in graph
            for node in networks[net]["graph"].nodes:
               # If node is in the week subset
               if node in list(df_week.author_id.unique()):
                  # Find the value for a given node in a given week, in the form "node": value (of emotion)
                  _[node] = df_week.loc[(df_week.author_id == node), "value"].values[0]
            # Append to the week array
            week_array.append(_)

         # Calculate distances from week to week
         for _ in range(len(week_array) - 1):

            distances.append((emotion, _, _ + 1, nd.ge(week_array[_], week_array[_ + 1], networks[net]["graph"], Q = networks[net]["Q"], normed = False)))
            distances_norm.append((emotion, _, _ + 1, nd.ge(week_array[_], week_array[_ + 1], networks[net]["graph"], Q = networks[net]["Q"], normed = True)))

      distances = pd.DataFrame(distances, columns=["Emotion", "From", "To", "Distance"])
      distances_norm = pd.DataFrame(distances_norm, columns=["Emotion", "From", "To", "Norm. Distance"])
      df = distances.merge(distances_norm, on=["Emotion", "From", "To"], how="inner")

      networks[net]["distances"] = df

      # Save to disk
      networks[net]["distances"].to_csv(f"{propagation_path}{net}-distances.csv", encoding="utf-8", index=False)
      
   # Else we load the Propagation Data
   else:
      networks[net]["distances"] = pd.read_csv(f"{propagation_path}{net}-distances.csv", encoding="utf-8", dtype={"Distance": float})

### Results per week

#### Non-normalized

Not being normalized, the propagation shows both structural changes in the network as well as propagation among nodes.

In [ ]:
for net in networks:
    sns.set(rc={'figure.figsize':(16,4)})
    sns.set_style("ticks")
    ax = sns.lineplot(data= networks[net]["distances"], x= "To", y= "Distance", hue= "Emotion", palette=emotions)
    plt.xlim(0)
    plt.ylim(0)
    plt.xlabel("Week")
    plt.grid("both", linestyle="dotted", color="grey", linewidth=0.5)
    sns.despine()
    plt.title(net)
    plt.legend("")
    plt.show()

#### Emotion Presence

##### Boxplots

In [ ]:
# For each network
for net in networks:

    print(net.upper())

    # Melt emotion columns to show boxplots
    temp = networks[net]["emotions-interpolated"].melt(value_vars=["optimism", "joy", "anger", "sadness"])
    # Plot settings
    sns.set(rc={'figure.figsize':(16,2)})
    sns.set_style("ticks")
    sns.boxplot(data = temp, y="variable", x="value", palette=emotions)
    plt.xlim(-0.01,1.01)
    plt.xlabel("")
    plt.ylabel("")
    plt.grid(axis="x", linestyle="dotted", color="grey", linewidth=0.5)
    sns.despine()
    plt.show()


##### Emotion presence avg, std, and quartiles

In [ ]:
for net in networks:
    print(f"NETWORK {net.upper()}")
    temp = networks[net]["emotions-interpolated"][emotions].describe()
    temp.drop("count", inplace=True)
    display(temp.T)

#### Testing for significant diffirences between the networks
We first use a Levene test to see if the data sets are suitable for an ANOVA test.

In [ ]:
for net in networks:
    print(f"Network {net.upper()}")
    for e in emotions:
        stat, p_value =kruskal(*[networks[net]['emotions-interpolated'][e] for net in networks])
        print(f"Emotion {e.capitalize()} - P-Value: {p_value} - {p_value / (0.05 * 4)}")

#### Distances per Network - Normalized

By being normalized, the propagation shows only the propagation proportionally among nodes.

In [ ]:
for net in networks.keys():
    sns.set(rc={'figure.figsize':(16,4)})
    sns.set_style("ticks")
    sns.lineplot(data= networks[net]["distances"], x= "To", y= "Norm. Distance", hue= "Emotion", palette=emotions)
    plt.xlim(0)
    plt.ylim(0)
    plt.legend("")
    plt.xlabel("")
    plt.grid("both", linestyle="dotted", color="grey", linewidth=0.5)
    sns.despine()
    plt.title(net)
    plt.show()

#### Propagation speed per network, avg. and std. for both Normalized and non-Normalized

In [ ]:
for net in networks:
    print(f"NETWORK {net.upper()}")
    display(networks[net]["distances"].loc[:,["Emotion", "Distance", "Norm. Distance"]].groupby("Emotion").agg(["mean", "std"]).reset_index().\
            sort_values(("Norm. Distance", "std"), ascending = False))

#### Distance Scoreboard (top 10)

In [ ]:
for net in networks:
    print(net.upper())
    display(networks[net]["distances"].sort_values(by = "Norm. Distance", ascending = False).head(10))

#### Emotion Acceleration (delta Speed)

In [ ]:
for net in networks:

    speed_changes = networks[net]["distances"].copy()
    dfs = []
    for e in emotions:
        _ = speed_changes.loc[speed_changes.Emotion == e]
        _["change"] = _["Norm. Distance"].diff()
        dfs.append(_)
    speed_changes = pd.concat(dfs)

    sns.set(rc={'figure.figsize':(16,4)})
    sns.set_style("ticks")
    ax = sns.lineplot(data= speed_changes, x= "To", y= "change", hue= "Emotion", palette=emotions)
    plt.xlim(0)
    plt.xlabel("Week")
    plt.ylabel("Acceleration")
    plt.legend("")
    plt.grid("both", linestyle="dotted", color="grey", linewidth=0.5)
    sns.despine()
    plt.title(net)
    plt.show()

    print(net)
    display(speed_changes.sort_values("change", ascending=False).head(10))

## Correlation between emotion presence and emotion variance against emotion propagation

### Emotion presence vs. Emotion Distance

We look at the correlation between the presence of emotions at the begining of the week and the propagation between that week and the next.

In [ ]:
print("EMOTION PRESENCE AT T AND DISTANCE FROM T - T+1")

for net in networks:

    print(f"\nNetwork: {net.upper()}")

    for e in emotions:

        emo_vector = networks[net]["emotions-interpolated"].groupby("week").sum().reset_index().loc[:49,e]
        emo_distance = networks[net]["distances"][networks[net]["distances"].Emotion == e]["Distance"]
        corr, pvalue = pearsonr(emo_vector, emo_distance)

        print(f"{e.capitalize()}: {corr:.3f} - P-Value: {pvalue:.5f}")

### Emotion variance vs. Emotion Distance

We look at the correlation between the variance of emotions from week to week and the propagation between those weeks.

In [ ]:
print("EMOTION DISTANCE FROM T - T+1 AND ITS DISTANCE")

for net in networks:

    print(f"\nNetwork: {net.upper()}")

    for e in emotions:

        emo_diff = networks[net]["emotions-interpolated"].groupby("week").mean()[e].diff().dropna()
        emo_distance = networks[net]["distances"][networks[net]["distances"].Emotion == e]["Distance"]
        corr, pvalue = pearsonr(emo_diff, emo_distance)

        print(f"{e.capitalize()}: {corr:.3f} - P-Value: {pvalue:.5f}")

### Spearman correlation between week number and emotion presence

We wish to inspect if there is a general trend in emotion presence throughout the year.

In [ ]:
threshold = 0.05 / 12

for net in networks:
    for e in emotions:
        c, p = pearsonr([i for i in range(51)], networks[net]["emotions-interpolated"].groupby("week").mean()[e])
        print(f"{net.upper()} - {e} - Statistic: {c:.3f}, p-value: {p:.5f} - {p < threshold}")
    print()

# Other Results & Resources

##### Avg. and Std. of speeds for each network - All weeks

In [ ]:
for net in networks:
    print(net.upper())
    display(networks[net]["distances"][["Emotion", "Distance", "Norm. Distance"]].groupby("Emotion").mean())

#### Example of propagation in small network:
- 5 Time steps
- 9 Nodes
- No edge weights

#### Examining the peaks

In [ ]:
# Network to examine
net = "us_congressmen"
# Week to look at
# None if all period
w = None
# Emotion to look at
# None if all emotions
e = None

if w != None:
    begin= (pd.to_datetime("2022-01-01")) + timedelta(7*w)
    end = (pd.to_datetime("2022-01-01")) + timedelta(7*(w+1)-1)
else:
    begin = pd.to_datetime("2022-01-01")
    end = pd.to_datetime("2022-12-23")

print(f"We are looking at:\n{net.upper()}\n{e if e != None else 'All emotions'}\nEvents between {begin} and {end}")

#### Numbers for the peaks

In [ ]:
if access_to_all:
    see_sub_grup_weekly_change(networks['universities']['sub_group'], 14, 'anger', 2, net = 'universities')
    see_sub_grup_weekly_change(networks['universities']['sub_group'], 32, 'anger', 2, net = 'universities')
    see_sub_grup_weekly_change(networks['us_congressmen']['sub_group'], 36, 'optimism', 3, net = 'us_congressmen')
    see_sub_grup_weekly_change(networks['journalists']['sub_group'], 35, 'optimism', 2, net = 'journalists')
    see_sub_grup_weekly_change(networks['journalists']['sub_group'], 35, 'optimism', 2, net = 'journalists')

In [ ]:
def mean_change(odf, emotions):
    """Find the mean change across the network each day for each feeling."""
    df = odf.copy()
    df = df.sort_values(['author_id', 'week'])

    for emotion in emotions:

        df[emotion + '_change'] = df[emotion].diff().abs()

    return df[df.week != 0]


def absolute_change(net, emotion_order):
    """Find the absolute change across the network each day for each feeling."""
    overlap = [i % 51 != 0 for i in range(int(len(net)/4))][:-1]
    overlap[0] = True

    values = net.set_index(['author_id', 'emotion', 'week'])
    values.sort_index()


    moved = values.diff().abs().reset_index()
    moved = moved[moved['week'] != 50]
    final = moved.groupby(['emotion', 'week']).value.sum()
    final = final.reset_index()
    emotions = []
    for emotion in emotion_order:
        emotions.extend(final[final.emotion == emotion].value)

    return emotions

mean_change = mean_change(networks['us_congressmen']['emotions-interpolated'], list(emotions.keys()))
mean_changes = mean_change[list(emotions.keys()) + ['week']].groupby('week').mean()
print("The mean change for congressmen for week 36 is", mean_changes['optimism'][36])
print("The mean change for congressmen for all weeks is", mean_change['optimism'].mean())

### Test for inactivity in all networks

In [ ]:
for net in networks.keys():
    no_tweets = networks[net]['emotions-aggregated'][networks[net]['emotions-aggregated'].weekly_tweet_values == 0]
    low_tweets = networks[net]['emotions-aggregated'][networks[net]['emotions-aggregated'].weekly_tweet_values < 4]
    print(f"{net} is inactive in {len(no_tweets) / len(networks[net]['emotions-aggregated']) * 100}% of weeks")
    print(f"{net} have low activity in {len(low_tweets) / len(networks[net]['emotions-aggregated']) * 100}% of weeks")
